# Aggregate all the various counts and metadata

In [2]:
import numpy

import pandas as pd

Get variables from `snakemake`:

In [7]:
counts_and_coverage_files = snakemake.input.counts_and_coverage
mito_ref_info_csv = snakemake.input.mito_ref_info
metadata_csv = snakemake.input.metadata
read_counts_csv = snakemake.input.read_counts
sars2_ref_id = snakemake.params.sars2_ref_id
mito_genomes_to_keep = snakemake.params.mito_genomes_to_keep
mito_composition_filters = snakemake.params.mito_composition_filters
metagenomic_descriptions = snakemake.params.metagenomic_descriptions

Read information on mitochondrial reference genomes, and get species name as just first two words:

In [5]:
mito_ref_info = pd.read_csv(mito_ref_info_csv)

mito_ref_info

,id,species,common_name
0,KX964606.1,Erinaceus amurensis,Amur hedgehog
1,NC_012095.1,Sus scrofa domesticus,pig
2,AC_000022.2,Rattus norvegicus strain Wistar,brown rat
3,NC_001913.1,Oryctolagus cuniculus,rabbit
4,NC_002008.4,Canis lupus familiaris,dog
...,...,...,...
4166,NC_029381.1,Tlacuatzin canescens,NaN
4167,NC_026049.1,Pelusios castaneus voucher AHNU26080283,NaN
4168,NC_044441.1,Aphyosemion cognatum voucher AMNH-I-251785,NaN
4169,NC_051037.1,Cinclus mexicanus,NaN


Read metadata on samples.
For sample titles that have multiple sample names, append the name to the title:

In [6]:
metadata_all = pd.read_csv(metadata_csv)

# downsize to most relevant metadata to keep
metadata = (
    metadata_all
    # make sample titles unique if they correspond to multiple names
    .assign(
        n_sample_names=lambda x: (
            x.groupby("Sample title")["Sample name"].transform("nunique")
        ),
        sample=lambda x: x["Sample title"].where(
            x["n_sample_names"] == 1,
            x["Sample title"] + "_" + x["Sample name"],
        )
    )
    # rename and organize
    .rename(columns={"Public description": "description"})
    [
        [
            "Run accession",
            "sample",
            "Sample name",
            "Collection date",
            "description",
            "Isolation source",
        ]
    ]
)

Tally all the alignment counts:

In [22]:
counts_and_coverage = pd.concat(
    [
        (
            pd.read_csv(
                f,
                sep="\t",
                names=["alignment_reference", "aligned_reads", "covered_bases"],
                header=0,
            )
            .assign(**{"Run accession": os.path.splitext(os.path.basename(f))[0]})
        )
        for f in counts_and_coverage_files
    ],
    ignore_index=True,
)[["Run accession", "alignment_reference", "aligned_reads", "covered_bases"]]

# make sure all alignment references are SARS2 or a known mitochondrial genome
assert set(counts_and_coverage["alignment_reference"]).issubset(
    set(mito_ref_info["id"]).union([sars2_ref_id])
)

assert (
    len(counts_and_coverage)
    == len(counts_and_coverage.drop_duplicates())
    == counts_and_coverage["alignment_reference"].nunique() * counts_and_coverage["Run accession"].nunique()
)

counts_and_coverage

,Run accession,alignment_reference,aligned_reads,covered_bases
0,CRR710761,NC_045512v2,3,129
1,CRR710761,NC_021950.1,0,0
2,CRR710761,NC_046471.1,0,0
3,CRR710761,NC_029247.1,0,0
4,CRR710761,NC_007783.2,0,0
...,...,...,...,...
1643763,CRR711155,NC_026049.1,0,0
1643764,CRR711155,NC_044441.1,0,0
1643765,CRR711155,NC_051037.1,0,0
1643766,CRR711155,NC_060309.1,0,0


We expect some correlation between aligned reads and covered bases, check that is true among references with reasonable coverage:

In [42]:
read_vs_coverage_corr = (
    counts_and_coverage
    .groupby("alignment_reference")
    [["aligned_reads", "covered_bases"]]
    .corr(method="spearman")
    .reset_index()
    .query("level_1 == 'aligned_reads'")
    .drop(columns=["level_1", "aligned_reads"])
    .rename(columns={"covered_bases": "correlation"})
    .query("correlation.notnull()")
    .sort_values("correlation")
    .merge(
        counts_and_coverage
        .groupby("alignment_reference", as_index=False)
        .aggregate(total_reads=pd.NamedAgg("aligned_reads", "sum"))
    )
    .query("total_reads > 100")
    .reset_index(drop=True)
)

display(read_vs_coverage_corr)

assert read_vs_coverage_corr["correlation"].min() > 0.65, read_vs_coverage_corr

,alignment_reference,correlation,total_reads
0,NC_065160.1,0.712755,164
1,NC_026307.1,0.777050,108
2,NC_037513.1,0.799770,2920
3,NC_015238.2,0.816475,790
4,NC_009728.1,0.860335,371176
...,...,...,...
165,NC_003128.3,0.999999,145
166,NC_008066.1,1.000000,157882
167,NC_023962.1,1.000000,194
168,NC_007439.1,1.000000,139


Get the read and alignment counts:

In [20]:
read_counts = pd.read_csv(read_counts_csv)

## SARS-CoV-2 read counts
Now get the SARS-CoV-2 read counts as a percentage of all preprocessed reads.

We first do this by run:

In [24]:
sars2_aligned_by_run = (
    counts_and_coverage
    .query("alignment_reference == @sars2_ref_id")
    .drop(columns="alignment_reference")
    .rename(
        columns={
            "aligned_reads": "SARS2_aligned_reads",
            "covered_bases": "SARS2_covered_bases",
        },
    )
    .merge(read_counts, validate="one_to_one", how="outer")
    .assign(
        percent_preprocessed_reads_aligning_to_SARS2=(
            lambda x: x["SARS2_aligned_reads"] / x["preprocessed_reads"] * 100
        ),
    )
    .merge(
        metadata,
        on="Run accession",
        validate="one_to_one",
        how="outer",
    )
)

sars2_aligned_by_run.to_csv(
    snakemake.output["sars2_aligned_by_run"], index=False, float_format="%.5g",
)

sars2_aligned_by_run

,Run accession,SARS2_aligned_reads,SARS2_covered_bases,total_reads,preprocessed_reads,percent_preprocessed_reads_aligning_to_SARS2,sample,Sample name,Collection date,description,Isolation source
0,CRR710761,3,129,106545428,90854684,3.301976e-06,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
1,CRR710762,1,43,213214887,206952411,4.832029e-07,Q61,Env_0576,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
2,CRR710763,3,129,120407578,120404558,2.491600e-06,Q64,Env_0579,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
3,CRR710764,2,86,110003167,87071974,2.296950e-06,Q68,Env_0583,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
4,CRR710765,2,86,106780574,106777962,1.873046e-06,Q69,Env_0584,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
...,...,...,...,...,...,...,...,...,...,...,...
389,CRR711142,0,0,123643938,123541212,0.000000e+00,CCDC-760,Env_0909,2020-03-02,RNA sequencing of total nucleic acids from env...,West Wine of HSM
390,CRR711152,594725,29870,3342164,3202808,1.856886e+01,F13-20,Env_0313_seq04,2020-01-20,SARS-CoV-2 Amplicon based SARS-CoV-2 whole gen...,West Wine of HSM
391,CRR711153,1210978,29869,5222964,5016616,2.413934e+01,F13-21,Env_0313_seq05,2020-01-21,SARS-CoV-2 Amplicon based SARS-CoV-2 whole gen...,West Wine of HSM
392,CRR711154,4156951,29870,4419724,4387884,9.473703e+01,B5-P3,Env_0126_seq06,2020-02-29,SARS-CoV-2 Amplicon based SARS-CoV-2 whole gen...,West Wine of HSM


Now we do this by sample (aggregating runs for each sample) for **just the metagenomic samples**.

In [26]:
# columns that we sum across runs for each sample
sum_cols = [
    "total_reads",
    "preprocessed_reads",
    "SARS2_aligned_reads",
]

sars2_aligned_by_metagenomic_sample = (
    sars2_aligned_by_run
    .query("description in @metagenomic_descriptions")
    .groupby(
        [
            c for c in sars2_aligned_by_run
            if c not in sum_cols
            and not c.startswith("percent_")
            and c not in ["Run accession", "SARS2_covered_bases"]
        ],
        as_index=False,
        dropna=False,
    )
    .aggregate({c: "sum" for c in sum_cols})
    .assign(
        percent_preprocessed_reads_aligning_to_SARS2=(
            lambda x: x["SARS2_aligned_reads"] / x["preprocessed_reads"] * 100
        ),
    )
)

dup_samples = (
    sars2_aligned_by_metagenomic_sample
    .groupby("sample")
    .aggregate(
        n=pd.NamedAgg("total_reads", "count"),
        sample_names=pd.NamedAgg("Sample name", "unique"),
    )
    .query("n > 1")
)

assert not len(dup_samples), f"Some duplicated sample names\n{dup_samples}"

sars2_aligned_by_metagenomic_sample.to_csv(
    snakemake.output["sars2_aligned_by_metagenomic_sample"], index=False, float_format="%.5g",
)

sars2_aligned_by_metagenomic_sample.sort_values("SARS2_aligned_reads")

,sample,Sample name,Collection date,description,Isolation source,total_reads,preprocessed_reads,SARS2_aligned_reads,percent_preprocessed_reads_aligning_to_SARS2
0,06-29-abv1,Env_0620,2020-01-23,RNA sequencing of total nucleic acids from env...,West Wine of HSM,1079014964,851849944,0,0.000000
102,HJ200033-20200112-1,Env_0548,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM,116117847,116115128,0,0.000000
103,HJ200034-20200112-1,Env_0549,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM,148630458,148628067,0,0.000000
104,HJ200035-20200112-1,Env_0550,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM,114656800,71848972,0,0.000000
105,HJ200036-20200112-1,Env_0551,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM,229948912,223587184,0,0.000000
...,...,...,...,...,...,...,...,...,...
69,F98,Env_0398,2020-01-01,RNA sequencing of total nucleic acids from env...,West Wine of HSM,401129274,399741992,330,0.000083
40,A61,Env_0061,2020-01-01,RNA sequencing of total nucleic acids from env...,West Wine of HSM,449907818,447903414,333,0.000074
46,B17,Env_0138,2020-01-01,RNA sequencing of total nucleic acids from env...,West Wine of HSM,213123774,212249334,405,0.000191
47,B5,Env_0126,2020-01-01,RNA sequencing of total nucleic acids from env...,West Wine of HSM,229464470,228684372,1458,0.000638


## Composition of mitochondrial reads

Do some filtering on the runs:
 - Get just the mitochondrial counts
 - Only keep runs with the metagenomic description
 - Exclude runs with insufficient alignment counts to mitochondria

In [47]:
mito_counts = (
    counts_and_coverage
    .merge(metadata, validate="many_to_one")
    .query("alignment_reference != @sars2_ref_id")
    .query("description in @metagenomic_descriptions")
)

insufficient_mito_reads = (
    mito_counts
    .groupby("Run accession", as_index=False)
    .aggregate(total_aligned_reads=pd.NamedAgg("aligned_reads", "sum"))
    .sort_values("total_aligned_reads")
    .query("total_aligned_reads < @mito_composition_filters['min_alignments_run_filter']")
    .reset_index(drop=True)
    .merge(metadata, how="left")
)

print("Excluding the following metagenomic runs with insufficient aligned mitochondrial reads:")
display(insufficient_mito_reads)

mito_counts = mito_counts[
    ~mito_counts["Run accession"].isin(insufficient_mito_reads["Run accession"])
]

Excluding the following metagenomic runs with insufficient aligned mitochondrial reads:


,Run accession,total_aligned_reads,sample,Sample name,Collection date,description,Isolation source
0,CRR710915,0,1-29-4,Env_0682,2020-01-29,RNA sequencing of total nucleic acids from env...,East Wine of HSM (Sewers or sewerage wells)
1,CRR710914,0,1-27-37,Env_0664,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells)
2,CRR711110,13,C9,Env_0839,2020-02-20,RNA sequencing of total nucleic acids from env...,West Wine of HSM
3,CRR710920,15,8-25-D_Env_0717,Env_0717,2020-02-03,RNA sequencing of total nucleic acids from env...,West Wine of HSM
4,CRR710919,16,8-25-D_Env_0717,Env_0717,2020-02-03,RNA sequencing of total nucleic acids from env...,West Wine of HSM
...,...,...,...,...,...,...,...
206,CRR711046,940,1-29-23,Env_0701,2020-01-29,RNA sequencing of total nucleic acids from env...,East Wine of HSM
207,CRR710950,959,HJ200015-20200112-1,Env_0530,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
208,CRR710925,968,HXJC4-1-3,Env_0802,2020-02-09,RNA sequencing of total nucleic acids from env...,Sewerage well in the surrounding areas
209,CRR710939,978,HJ200004-20200112-1,Env_0519,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM


Now exclude mitochondrial reference genomes that don't have a high enough percentage of read alignments or are not delineated in the list to keep:

In [51]:
mito_max_primary_percent = ( 
    mito_counts
    .assign(
        total=lambda x: x.groupby("Run accession")["aligned_reads"].transform("sum"),
        percent=lambda x: 100 * x["aligned_reads"] / x["total"],
    )
    .groupby("alignment_reference", as_index=False)
    .aggregate(
        max_percent=pd.NamedAgg("percent", "max"),
        avg_percent=pd.NamedAgg("percent", "mean"),
        max_coverage=pd.NamedAgg("covered_bases", "max"),
    )
    .sort_values("max_percent")
    .merge(mito_ref_info.rename(columns={"id": "alignment_reference"}))
)

min_percent = mito_composition_filters["min_percent_aligned_genome_filter"]
mito_to_keep = (
    mito_max_primary_percent
    .assign(
        reason_kept=lambda x: x.apply(
            lambda row: (
                "specified to keep in config"
                if any(row["species"].startswith(s) for s in mito_genomes_to_keep.values())
                else (
                    f"at least {min_percent}% in one run"
                    if row["max_percent"] >= min_percent
                    else
                    "not retained"
                )
            ),
            axis=1,
        ),
    )
    .query("reason_kept != 'not retained'")
    .sort_values("max_percent", ascending=False)
    .reset_index(drop=True)
)

assert all(
    any(s_kept.startswith(s_to_keep) for s_kept in mito_to_keep["species"])
    for s_to_keep in mito_genomes_to_keep.values()
)

print("Keeping the following mitochondrial genomes:")
display(mito_to_keep.sort_values("avg_percent", ascending=False).round(2))

mito_ids_to_keep = mito_to_keep["alignment_reference"].tolist()

Keeping the following mitochondrial genomes:


,alignment_reference,max_percent,avg_percent,max_coverage,species,common_name,reason_kept
0,NC_012920.1,99.01,14.72,14000,Homo sapiens,human,specified to keep in config
2,NC_009728.1,98.63,14.40,73,Osteolaemus tetraspis,dwarf crocodile,specified to keep in config
3,NC_040902.1,97.71,11.26,16784,Gallus gallus spadiceus,chicken,specified to keep in config
12,NC_022418.1,63.96,6.36,16567,Anas poecilorhyncha,spot-billed duck,specified to keep in config
5,NC_030041.1,87.17,3.91,13056,Ptyas mucosa,Oriental rat snake,specified to keep in config
7,NC_020011.1,77.69,3.58,15491,Channa maculata,snakehead fish,specified to keep in config
11,NC_010288.1,68.76,3.35,16548,Ctenopharyngodon idella,carp,specified to keep in config
9,NC_013700.1,75.49,2.82,15490,Nyctereutes procyonoides,raccoon dog,specified to keep in config
4,NC_006853.1,94.65,2.78,16338,Bos taurus,cow,specified to keep in config
6,NC_002008.4,80.41,2.52,14587,Canis lupus familiaris,dog,specified to keep in config


Now get just the counts for those mitochondrial genomes to keep, aggregating all other counts to "other" and pivoting / melting to add zero counts:

In [68]:
mito_counts_by_metagenomic_run = (
    counts_and_coverage
    [["Run accession", "alignment_reference", "aligned_reads"]]
    # aggregate all other counts as "other"
    .assign(
        alignment_reference=lambda x: x["alignment_reference"].where(
            x["alignment_reference"].isin(mito_ids_to_keep), "other"
        ),
    )
    .groupby(["Run accession", "alignment_reference"], as_index=False)
    .aggregate(aligned_reads=pd.NamedAgg("aligned_reads", "sum"))
    .rename(columns={"alignment_reference": "reference_id"})
    # add more information on references, and shorten species name to first two words
    .merge(mito_ref_info.rename(columns={"id": "reference_id"}), how="left")
    .assign(
        species=lambda x: numpy.where(
            x["reference_id"] == "other",
            "other",
            x["species"].map(lambda s: " ".join(s.split()[: 2]) if not pd.isnull(s) else s),
        ),
        common_name=lambda x: x["common_name"].where(
            x["reference_id"] != "other", "other",
        ),
    )
    # add other information
    .merge(
        sars2_aligned_by_run
        .drop(columns=["percent_preprocessed_reads_aligning_to_SARS2", "SARS2_covered_bases"]),
        how="left",
    )
)

mito_counts_by_metagenomic_run.to_csv(
    snakemake.output["mito_composition_by_metagenomic_run"], index=False, float_format="%.5f",
)

# debugging
assert mito_counts_by_metagenomic_run.drop(columns="common_name").notnull().all().all(), mito_counts_by_metagenomic_run.notnull().all()

mito_counts_by_metagenomic_run

,Run accession,reference_id,aligned_reads,species,common_name,SARS2_aligned_reads,total_reads,preprocessed_reads,sample,Sample name,Collection date,description,Isolation source
0,CRR710761,AC_000022.2,1,Rattus norvegicus,brown rat,3,106545428,90854684,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
1,CRR710761,KX964606.1,73,Erinaceus amurensis,Amur hedgehog,3,106545428,90854684,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
2,CRR710761,NC_001700.1,0,Felis catus,cat,3,106545428,90854684,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
3,CRR710761,NC_001913.1,0,Oryctolagus cuniculus,rabbit,3,106545428,90854684,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
4,CRR710761,NC_002008.4,1,Canis lupus,dog,3,106545428,90854684,Q37,Env_0552,2020-01-12,RNA sequencing of total nucleic acids from env...,West Wine of HSM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,CRR712210,NC_041068.1,0,Elaphe dione,rat snake,0,151171387,25617933,1-27-11,Env_0642,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells)
19302,CRR712210,NC_042233.1,2,Numenius tenuirostris,curlew (bird),0,151171387,25617933,1-27-11,Env_0642,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells)
19303,CRR712210,NC_050263.1,0,Hystrix brachyura,Malayan porcupine,0,151171387,25617933,1-27-11,Env_0642,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells)
19304,CRR712210,NC_069631.1,0,Terrapene carolina,box turtle,0,151171387,25617933,1-27-11,Env_0642,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells)


Now we get the mitochondrial counts by **sample** (aggregating counts across run for the same sample:

In [72]:
# columns that we sum across runs for each sample
sum_cols = [
    "aligned_reads",
    "total_reads",
    "preprocessed_reads",
    "SARS2_aligned_reads",
]

mito_counts_by_metagenomic_sample = (
    mito_counts_by_metagenomic_run
    .groupby(
        [
            c for c in mito_counts_by_metagenomic_run
            if c not in sum_cols
            and c not in {"Run accession"}
        ],
        as_index=False,
        dropna=False,
    )
    .aggregate({c: "sum" for c in sum_cols})
)

dup_samples = (
    mito_counts_by_metagenomic_sample
    .groupby(["reference_id", "sample"])
    .aggregate(
        n=pd.NamedAgg("aligned_reads", "count"),
        sample_names=pd.NamedAgg("Sample name", "unique"),
    )
    .query("n > 1")
)

assert not len(dup_samples), f"Some duplicated sample names\n{dup_samples}"

mito_counts_by_metagenomic_sample.to_csv(
    snakemake.output["mito_composition_by_metagenomic_sample"], index=False, float_format="%.5g",
)

mito_counts_by_metagenomic_sample

,reference_id,species,common_name,sample,Sample name,Collection date,description,Isolation source,aligned_reads,total_reads,preprocessed_reads,SARS2_aligned_reads
0,AC_000022.2,Rattus norvegicus,brown rat,06-29-abv1,Env_0620,2020-01-23,RNA sequencing of total nucleic acids from env...,West Wine of HSM,3,1079014964,851849944,0
1,AC_000022.2,Rattus norvegicus,brown rat,1-27-11,Env_0642,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),8,821800418,571936627,0
2,AC_000022.2,Rattus norvegicus,brown rat,1-27-12,Env_0643,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),3,2038484394,2029662756,0
3,AC_000022.2,Rattus norvegicus,brown rat,1-27-28,Env_0657,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),0,290477942,289351106,0
4,AC_000022.2,Rattus norvegicus,brown rat,1-27-33,Env_0660,2020-01-27,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),0,92724742,92467336,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8619,other,other,other,YCHC2-1-2,Env_0798,2020-02-09,RNA sequencing of total nucleic acids from env...,Sewerage well in the surrounding areas,36,868053694,865532062,0
8620,other,other,other,YCHC2-1-3,Env_0799,2020-02-09,RNA sequencing of total nucleic acids from env...,Sewerage well in the surrounding areas,11,662632390,660721662,0
8621,other,other,other,w-6-29-33,Env_0828,2020-02-15,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),77,367737968,366257533,0
8622,other,other,other,w-zong-1,Env_0829,2020-02-15,RNA sequencing of total nucleic acids from env...,West Wine of HSM (Sewers or sewerage wells),204,433163214,431193910,0
